In [5]:
pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 MB 26.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 54.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 52.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 43.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import cv2
import IPython
from IPython.display import clear_output

import sys 
import os
sys.path.append(os.path.abspath("../"))
from module.Segmentor import Segmentor
from module.Classifier import Classifier

import time

In [ ]:
filename = '../assets/test.mp4'

inputStream = cv2.VideoCapture(filename)

FPS = int(24)
inputWidth = int(inputStream.get(cv2.CAP_PROP_FRAME_WIDTH))
inputHeight = int(inputStream.get(cv2.CAP_PROP_FRAME_HEIGHT))

outputStream = cv2.VideoWriter('../assets/test-results.mp4',
                               cv2.VideoWriter_fourcc('x', '2', '6', '4'),
                               FPS, (inputWidth, inputHeight))

while inputStream.isOpened():
    ret, frame = inputStream.read()
    
    if not ret:
        print("End of stream? Exiting...")
        break
        
    segmentor = Segmentor(
        min_p=4,
        max_p=400,
        min_a=1000,
        max_a=20000,
        suit_x_ratio=2 / 7,
        suit_y_ratio=1 / 5,
    )
    
    classifer = Classifier()
    
    st = time.time()
    results = segmentor.seg(frame)
    en = time.time()
    print("time:", en - st)
    
    print(len(results))

    frameMarked = frame.copy()
    for result in results:
        contour = np.array(result['bbox'])
        cv2.drawContours(frameMarked, [contour], 0, (0, 0, 255), 3)
        card_class = classifer.get_class(result['suit'])
        cv2.putText(frameMarked, card_class, [contour[0]], cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255))
    
    # Encode image as jpg numpy array
    _, buf = cv2.imencode(".jpg", frameMarked)
    # Draw result
    IPython.display.display(IPython.display.Image(data=buf))
    outputStream.write(frameMarked)
        
    clear_output(wait=True) 
    
    if cv2.waitKey(1) == ord('q'):
        break
        
outputStream.release()
inputStream.release()

In [4]:
print("test")

test


In [ ]:
print("hey")
filename = '../bicyclecard/images/IMG_20230520_185700449.jpg'
inputImage = cv2.imread(filename)

segmentor = Segmentor(
    min_p=4,
    max_p=400,
    min_a=1000,
    max_a=20000,
    suit_x_ratio=2 / 7,
    suit_y_ratio=1 / 5,
)

classifer = Classifier()
    
st = time.time()
results = segmentor.seg(inputImage)
en = time.time()
print("time:", en - st)

print(len(results))

inputImageMarked = inputImage.copy()
for result in results:
    contour = np.array(result['bbox'])
    cv2.drawContours(inputImageMarked, [contour], 0, (0, 0, 255), 3)
    card_class = classifer.get_class(result['suit'])
    cv2.putText(inputImageMarked, card_class, [contour[0]], cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255))
    
# Encode image as jpg numpy array
_, buf = cv2.imencode(".jpg", inputImageMarked)
# Draw result
IPython.display.display(IPython.display.Image(data=buf))